In [2]:
from transformers import AutoModelForImageClassification, AutoImageProcessor

repo_name = "JacobJan/swin-tiny-patch4-window7-224-finetuned-eurosat"

image_processor = AutoImageProcessor.from_pretrained(repo_name)
model = AutoModelForImageClassification.from_pretrained(repo_name)

In [7]:
# prepare image for the model
from PIL import Image
image = Image.open('test_dataset_gray/dressoir_mccp.jpg')
# encoding = image_processor(image.convert("RGB"), return_tensors="pt")
# print(encoding.pixel_values.shape)

In [4]:
import torch

# forward pass
with torch.no_grad():
    outputs = model(**encoding)
    logits = outputs.logits

In [5]:
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

Predicted class: 0


In [41]:
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights
import os
import csv
import cv2

directory = os.fsencode('scraped_images')

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()
labeled_list = [[]]

for file in os.listdir('scraped_images_grayscaled_big'):
    # Step 3: Apply inference preprocessing transforms
    # image = cv2.imread(os.path.join('scraped_images', file))
    # if image.shape[0] == 1:
    #     image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
    # image = read_image(os.path.join('scraped_images', file)).convert('RGB')
    image = Image.open(os.path.join('scraped_images_grayscaled_big', file)).convert('RGB')
    batch = preprocess(image).unsqueeze(0)

    # Step 4: Use the model and print the predicted category
    prediction = model(batch).squeeze(0).softmax(0)
    class_id = prediction.argmax().item()
    score = prediction[class_id].item()
    category_name = weights.meta["categories"][class_id]
    labeled_list += [file, category_name]

with open('image_categories.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(labeled_list)

In [42]:
import pandas as pd
print(labeled_list[2])

combined_list = []

# Iterate through the list starting from the second item, in steps of 2
for i in range(1, len(labeled_list) - 1, 2):
    combined_list.append([labeled_list[i], labeled_list[i + 1]])

# If the list length is odd, we need to handle the last item
if len(labeled_list) % 2 == 0:
    combined_list.append([labeled_list[-2], labeled_list[-1]])
else:
    combined_list.append([labeled_list[-3], labeled_list[-2]])

print(combined_list[1])
my_df = pd.DataFrame(combined_list)

my_df.to_csv('my_csv.csv', index=False, header=False)

altar
['00100_524_id=cp168205_badv.jpg', 'airship']
